In [2]:
import sys

# 強制在 Jupyter 當前使用的 python.exe 下安裝 torch
!{sys.executable} -m pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu
!{sys.executable} -m pip install transformers datasets

Looking in indexes: https://download.pytorch.org/whl/cpu


In [3]:
import sys
print(sys.executable)

C:\Users\USER\anaconda3\python.exe


In [ ]:
import torch
import transformers
from datasets import load_dataset

print("✅ Torch 安裝成功！版本:", torch.__version__)
print("✅ Transformers 安裝成功！版本:", transformers.__version__)
print("✅ 是否可用 GPU:", torch.cuda.is_available())

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
import pandas as pd
import time

# 1. 採樣極小量測試 (先確認能跑通)
df_bert = df.groupby('label').apply(lambda x: x.sample(n=100)).reset_index(drop=True)

# 2. 改用 DistilBERT (更輕量)
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=64) # 長度縮短

dataset = Dataset.from_pandas(df_bert[['text', 'label']])
tokenized_datasets = dataset.map(tokenize_function, batched=True)
train_test = tokenized_datasets.train_test_split(test_size=0.2)

# 3. 載入模型
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

# 4. 極度保守的訓練參數
training_args = TrainingArguments(
    output_dir="./bert_test",
    evaluation_strategy="epoch",
    num_train_epochs=1,
    per_device_train_batch_size=1, # 設為 1，最省記憶體
    logging_steps=10,
    use_cpu=True 
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_test["train"],
    eval_dataset=train_test["test"]
)

# 5. 開始訓練
try:
    start_time = time.time()
    trainer.train()
    print(f"訓練成功！耗時: {time.time() - start_time:.2f} 秒")
except Exception as e:
    print(f"還是失敗了，錯誤訊息: {e}")